In [4]:
# Retrait du dataset

import pandas as pd
raw_data = pd.read_csv('Ressources' + '/'+ 'Dataset_feuilles_1.csv')
test_data = pd.read_csv('Ressources' + '/'+ 'test.csv')

In [14]:
# Conversion des lettres en chiffres

from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
for col in raw_data.columns:
    raw_data[col] = labelencoder.fit_transform(raw_data[col])

for col in test_data.columns:
    test_data[col] = labelencoder.fit_transform(test_data[col])
    
test_data

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0,10,5,40,6,2,8,3,0,3,...,7,0,16,1,16,0,0,0,4,55
1,1,4,3,33,5,2,7,4,0,16,...,0,0,7,2,14,0,0,1,38,46
2,2,0,0,1,11,21,0,12,0,6,...,98,0,1,0,0,0,0,16,0,0
3,3,0,0,5,6,9,0,2,0,2,...,13,16,3,35,14,0,0,75,0,9
4,4,1,0,8,5,20,0,5,0,3,...,0,32,17,11,42,0,0,8,10,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,589,0,0,2,8,21,0,9,0,3,...,79,0,5,0,4,0,0,19,0,1
590,590,0,2,2,3,9,0,9,3,0,...,13,5,5,3,10,0,0,76,0,17
591,591,9,15,8,7,2,8,13,0,0,...,63,0,29,0,2,0,0,0,43,7
592,592,7,5,31,13,18,13,20,0,2,...,4,0,1,0,12,0,0,0,12,19


In [7]:
# Separation en donnees test et entrainement

from sklearn.model_selection import train_test_split

# On récupère les features d'un côté...
X = raw_data.iloc[:,2:194]

# et les labels de l'autre
y = raw_data.iloc[:,1] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

### Regression Logistique ###

### SVM Lineaire ###

In [20]:
# Test des performances lineaires

import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression

svm = LinearSVC()
lr = LogisticRegression(solver = 'liblinear')

params = { 'C': np.logspace(-3, 3, 7) }

gs_svm = GridSearchCV(lr, params, cv=3)
gs_svm.fit(X_train, y_train)

print(gs_svm.best_params_)

{'C': 0.01}


In [ ]:
# Passage a l'evaluation

from sklearn.metrics import roc_curve, auc

lr = LogisticRegression(multi_class = 'ovr')
lr.fit(X_train,y_train)
y_prob = lr.predict_proba(X_test)[:,1] 

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)


print(roc_auc)

/home/pc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pc/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

In [23]:
help(LogisticRegression)

Help on class LogisticRegression in module sklearn.linear_model._logistic:

class LogisticRegression(sklearn.base.BaseEstimator, sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin)
 |  LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
 |  
 |  Logistic Regression (aka logit, MaxEnt) classifier.
 |  
 |  In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
 |  scheme if the 'multi_class' option is set to 'ovr', and uses the
 |  cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
 |  (Currently the 'multinomial' option is supported only by the 'lbfgs',
 |  'sag', 'saga' and 'newton-cg' solvers.)
 |  
 |  This class implements regularized logistic regression using the
 |  'liblinear' library, 'newton-cg', 's